In [11]:
import os
import cv2
import math
import numpy as np
from typing import Tuple, List

os: για διαχείριση φακέλων/paths (π.χ. os.makedirs, os.path.join).

cv2: βασικές λειτουργίες OpenCV (ανάγνωση/γραφή εικόνας, threshold, morphology, connectedComponents).

math: για ceil κτλ.

numpy: βασικός χειρισμός πινάκων/εικόνων.

typing: μόνο για type hints, ώστε ο κώδικας να είναι “καθαρός” και πιο τεκμηριωμένος.

In [12]:
def save_img(path: str, img: np.ndarray) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    cv2.imwrite(path, img)


In [13]:
def to_uint8(img: np.ndarray) -> np.ndarray:
    return np.clip(np.rint(img), 0, 255).astype(np.uint8)


In [15]:
def add_gaussian_noise(img: np.ndarray, mean: float = 0.0, sigma: float = 15.0) -> np.ndarray:
    """
    Προσθέτει Gaussian (κανονικό) θόρυβο σε μια εικόνα.

    Παραμέτροι
    ----------
    img : np.ndarray
        Η αρχική εικόνα ως πίνακας NumPy. Συνήθως περιέχει τιμές pixel στο εύρος [0, 255].
    mean : float, προεπιλογή 0.0
        Η μέση τιμή (μ) της κανονικής κατανομής του θορύβου.
        Αν είναι 0, ο θόρυβος δεν «σκοτώνει» γενικά τη φωτεινότητα, απλώς την ταλαντώνει γύρω από την αρχική.
    sigma : float, προεπιλογή 15.0
        Η τυπική απόκλιση (σ) της κανονικής κατανομής.
        Όσο μεγαλύτερη η σ, τόσο πιο έντονος (ισχυρός) είναι ο θόρυβος.

    Επιστρέφει
    ----------
    np.ndarray
        Την «θορυβώδη» εικόνα, με τιμές pixel στον τυπικό χώρο [0, 255] και τύπο uint8.
    """

    # Δημιουργεί έναν πίνακα "noise" με τις ίδιες διαστάσεις (shape) όπως η εικόνα "img".
    # Οι τιμές του προέρχονται από κανονική (Gaussian) κατανομή με:
    #   - μέση τιμή "mean"
    #   - τυπική απόκλιση "sigma"
    #
    # np.random.normal(mean, sigma, img.shape):
    #   - mean  : κέντρο της κατανομής (που τείνουν να συγκεντρώνονται οι τιμές)
    #   - sigma : πόσο απλωμένες είναι οι τιμές γύρω από το mean
    #   - img.shape : ορίζει ότι ο θόρυβος θα έχει ακριβώς τόσες θέσεις όσα και τα pixel της εικόνας.
    noise = np.random.normal(mean, sigma, img.shape)

    # Μετατρέπει προσωρινά την εικόνα σε float64 πριν προσθέσουμε τον θόρυβο.
    # Γιατί;
    #   - Οι εικόνες συνήθως είναι τύπου uint8 (ακέραιοι 0–255).
    #   - Αν προσθέταμε απευθείας θόρυβο σε uint8, θα μπορούσαν να συμβούν υπερχειλίσεις (overflow)
    #     ή απώλεια δεκαδικών.
    #   - Με το float64 επιτρέπουμε:
    #       * δεκαδικές τιμές κατά τη διάρκεια του υπολογισμού
    #       * αρνητικά ή >255 προσωρινά, ώστε η πράξη να είναι μαθηματικά σωστή.
    #
    # Στη συνέχεια προσθέτουμε τον θόρυβο "noise" στοιχείο–προς–στοιχείο (pixel–προς–pixel).
    noisy = img.astype(np.float64) + noise

    # Εδώ καλούμε τη βοηθητική συνάρτηση to_uint8(noisy),
    # η οποία:
    #   1. Στρογγυλοποιεί τις τιμές στο κοντινότερο ακέραιο.
    #   2. «Κόβει» (clip) ό,τι είναι κάτω από 0 στο 0 και ό,τι είναι πάνω από 255 στο 255.
    #   3. Μετατρέπει τον πίνακα πίσω σε τύπο uint8, που είναι ο τυπικός τύπος για εικόνες.
    #
    # Έτσι, το τελικό αποτέλεσμα είναι μια έγκυρη εικόνα που μπορεί να εμφανιστεί ή να αποθηκευτεί κανονικά.
    return to_uint8(noisy)


In [16]:
def add_salt_pepper(img: np.ndarray, amount: float = 0.01, s_vs_p: float = 0.5) -> np.ndarray:
    """
    Προσθέτει θόρυβο τύπου 'salt and pepper' (δηλαδή τυχαία λευκά και μαύρα pixel) σε μια εικόνα.

    Παραμέτροι
    ----------
    img : np.ndarray
        Η αρχική εικόνα σε μορφή πίνακα NumPy.
    amount : float, προεπιλογή 0.01
        Το ποσοστό των pixel της εικόνας που θα αλλοιωθούν από τον θόρυβο.
        Π.χ. 0.01 σημαίνει ότι θα αλλοιωθεί περίπου το 1% των pixel.
    s_vs_p : float, προεπιλογή 0.5
        Ο λόγος ανάμεσα στα “salt” (λευκά pixel) και “pepper” (μαύρα pixel).
        Π.χ. 0.5 σημαίνει ίσος αριθμός λευκών και μαύρων σημείων.

    Επιστρέφει
    ----------
    np.ndarray
        Την εικόνα με προστιθέμενο θόρυβο salt & pepper.
    """

    # Δημιουργούμε ένα αντίγραφο της αρχικής εικόνας για να μην αλλάξει το πρωτότυπο.
    noisy = img.copy()

    # Υπολογίζουμε τον συνολικό αριθμό των pixel της εικόνας.
    num_pixels = img.size  # π.χ. αν η εικόνα είναι 512x512 → 262,144 pixel

    # Υπολογίζουμε πόσα pixel θα γίνουν "salt" (λευκά)
    # και πόσα "pepper" (μαύρα), με βάση το ποσοστό 'amount' και τον λόγο 's_vs_p'.
    num_salt = int(math.ceil(amount * num_pixels * s_vs_p))
    num_pepper = int(math.ceil(amount * num_pixels * (1.0 - s_vs_p)))

    # === Προσθήκη λευκών pixel ("salt") ===
    # Δημιουργούμε τυχαίες συντεταγμένες για τα pixel που θα γίνουν λευκά.
    # np.random.randint(0, img.shape[0], num_salt) → τυχαίες γραμμές (ύψος)
    # np.random.randint(0, img.shape[1], num_salt) → τυχαίες στήλες (πλάτος)
    coords = (np.random.randint(0, img.shape[0], num_salt),
              np.random.randint(0, img.shape[1], num_salt))

    # Θέτουμε αυτά τα pixel στη μέγιστη τιμή φωτεινότητας (255 → λευκό)
    noisy[coords] = 255

    # === Προσθήκη μαύρων pixel ("pepper") ===
    # Δημιουργούμε νέες τυχαίες συντεταγμένες για τα pixel που θα γίνουν μαύρα.
    coords = (np.random.randint(0, img.shape[0], num_pepper),
              np.random.randint(0, img.shape[1], num_pepper))

    # Θέτουμε αυτά τα pixel στη χαμηλότερη τιμή φωτεινότητας (0 → μαύρο)
    noisy[coords] = 0

    # Επιστρέφουμε την εικόνα με τον θόρυβο salt & pepper
    return noisy


Η συνάρτηση παίρνει μια καθαρή εικόνα και «ρίχνει πάνω της» τυχαίες μαύρες και λευκές κουκίδες

s_vs_p: αναλογία salt (λευκά) vs pepper (μαύρα). Στο 0.5 είναι 50–50.

In [17]:
def pad_reflect(img: np.ndarray, k: int) -> np.ndarray:
    """
    Επεκτείνει (περιβάλλει) μια εικόνα προσθέτοντας γύρω της «αντανάκλαση» των άκρων της.
    Αυτό χρησιμοποιείται συχνά πριν από φιλτράρισμα ή άλλες πράξεις που χρειάζονται γειτονικά pixel.

    Παραμέτροι
    ----------
    img : np.ndarray
        Η αρχική εικόνα σε μορφή πίνακα NumPy.
    k : int
        Το μέγεθος του φίλτρου ή του παραθύρου που θα χρησιμοποιηθεί (π.χ. 3 για φίλτρο 3x3).

    Επιστρέφει
    ----------
    np.ndarray
        Μια νέα εικόνα με επιπλέον «περιθώριο» γύρω γύρω, που είναι κατοπτρικό αντίγραφο των άκρων της αρχικής.
    """

    # Υπολογίζουμε πόσα pixel padding χρειάζονται γύρω από την εικόνα.
    # Αν το φίλτρο έχει μέγεθος k, τότε προσθέτουμε r = k // 2 pixel σε κάθε πλευρά.
    # Για παράδειγμα, αν k = 3 → r = 1, δηλαδή προσθέτουμε 1 pixel πάνω, κάτω, δεξιά και αριστερά.
    r = k // 2

    # Εδώ γίνεται η "επέκταση" της εικόνας:
    # np.pad(img, ((r, r), (r, r)), mode='reflect') σημαίνει:
    #   - ((r, r), (r, r)) → προσθέτουμε r σειρές πάνω και κάτω, και r στήλες δεξιά και αριστερά.
    #   - mode='reflect' → τα νέα pixel δεν είναι τυχαία ή μαύρα, αλλά αντίγραφα των υπαρχόντων
    #     με καθρέφτισμα (αντανάκλαση).
    #
    # Παράδειγμα:
    # Αν το αριστερό άκρο έχει pixel [3, 5], το νέο padding θα είναι [5, 3 | 3, 5].
    return np.pad(img, ((r, r), (r, r)), mode='reflect')


Αυτή η συνάρτηση «μεγαλώνει» την εικόνα λίγο γύρω γύρω, αντιγράφοντας και καθρεφτίζοντας τα άκρα της. Αυτό βοηθάει ώστε, όταν εφαρμόζουμε φίλτρα (π.χ. blur ή edge detection), να μην έχουμε παραμορφώσεις στα όρια της εικόνας.

In [18]:
def mean_filter(img: np.ndarray, k: int = 3) -> np.ndarray:
    """
    Εφαρμόζει μέσο φίλτρο (mean filter / box filter) σε γκρίζα εικόνα.

    Κάθε pixel της εξόδου είναι ο μέσος όρος των τιμών μέσα σε ένα
    τετραγωνικό παράθυρο k × k γύρω από το αντίστοιχο pixel της εισόδου.
    """

    # Βεβαιωνόμαστε ότι το μέγεθος του παραθύρου k είναι:
    #   - περιττός αριθμός (π.χ. 3, 5, 7, ...) ώστε να υπάρχει "κέντρο"
    #   - τουλάχιστον 3 (δεν έχει νόημα φίλτρο 1x1)
    assert k % 2 == 1 and k >= 3

    # Πριν εφαρμόσουμε το φίλτρο, "μεγαλώνουμε" την εικόνα γύρω-γύρω
    # με αντανάκλαση των άκρων (reflect padding), ώστε:
    #   - να μπορούμε να πάρουμε πλήρες παράθυρο k×k ακόμη και στα σύνορα
    #   - να αποφύγουμε τεχνητές μαύρες/λευκές ζώνες στα άκρα.
    pad = pad_reflect(img, k)

    # r = "ακτίνα" του παραθύρου.
    # Αν k = 3 → r = 1 (1 pixel προς κάθε πλευρά από το κέντρο)
    # Αν k = 5 → r = 2 κ.ο.κ.
    r = k // 2

    # Δημιουργούμε έναν κενό πίνακα εξόδου με τις ίδιες διαστάσεις με την αρχική εικόνα.
    # Χρησιμοποιούμε float64 για να κάνουμε με ακρίβεια τους μέσους όρους (πριν γυρίσουμε σε uint8).
    out = np.empty_like(img, dtype=np.float64)

    # Υλοποίηση με "αθροιστική εικόνα" (integral image) για O(1) χρόνο ανά παράθυρο.
    #
    # Η integral_image(pad) δημιουργεί έναν πίνακα "integ" τέτοιο ώστε:
    #   - Κάθε στοιχείο (y, x) του integ να περιέχει το άθροισμα όλων των pixel
    #     της pad που βρίσκονται μέσα στο ορθογώνιο από (0, 0) έως (y, x).
    #
    # Με αυτόν τον τρόπο μπορούμε να υπολογίσουμε το άθροισμα οποιουδήποτε
    # ορθογωνίου με μερικές μόνο πράξεις αντί να κάνουμε διπλούς/τριπλούς βρόχους.
    integ = integral_image(pad.astype(np.float64))

    # Διαστάσεις της αρχικής (μη padded) εικόνας.
    H, W = img.shape

    # Διατρέχουμε όλα τα pixel της αρχικής εικόνας.
    for y in range(H):
        for x in range(W):
            # Για κάθε (y, x) της αρχικής εικόνας,
            # θέλουμε το αντίστοιχο παράθυρο k×k πάνω στην pad.
            #
            # Επειδή η pad είναι μεγαλύτερη (λόγω padding r pixel γύρω-γύρω),
            # το pixel (y, x) της img αντιστοιχεί στο (y + r, x + r) της pad.
            #
            # Όμως εδώ δουλεύουμε κατευθείαν σε συντεταγμένες της pad:
            #   - y0, x0 : πάνω-αριστερή γωνία του παραθύρου
            #   - y1, x1 : κάτω-δεξιά γωνία του παραθύρου
            #
            # Η περιοχή k×k γύρω από το (y + r, x + r) έχει:
            #   - κατακόρυφο εύρος [y, y + 2*r]
            #   - οριζόντιο  εύρος [x, x + 2*r]
            #
            # (γιατί έχουμε ήδη "ενσωματώσει" το r μέσω του pad_reflect)
            y0, x0 = y, x
            y1, x1 = y + 2 * r, x + 2 * r

            # Υπολογίζουμε το άθροισμα των τιμών pixel στο ορθογώνιο
            # [x0, x1] × [y0, y1] της padded εικόνας, χρησιμοποιώντας την
            # αθροιστική εικόνα "integ".
            #
            # Η rect_sum(integ, x0, y0, x1, y1) θεωρούμε ότι:
            #   - χρησιμοποιεί τον τύπο της integral image
            #   - επιστρέφει σε O(1) χρόνο το άθροισμα όλων των τιμών στο παράθυρο.
            s = rect_sum(integ, x0, y0, x1, y1)

            # Ο μέσος όρος στο παράθυρο k×k είναι:
            #   mean = (άθροισμα τιμών) / (αριθμός pixel στο παράθυρο)
            out[y, x] = s / (k * k)

    # Τέλος, μετατρέπουμε την εικόνα out σε έγκυρες τιμές εικόνας:
    #   - στρογγυλοποίηση
    #   - "κόψιμο" στο [0, 255]
    #   - μετατροπή σε uint8
    return to_uint8(out)


In [19]:
def median_filter(img: np.ndarray, k: int = 3) -> np.ndarray:
    """
    Εφαρμόζει φίλτρο διάμεσης τιμής (median filter) σε γκρίζα εικόνα.

    Το φίλτρο αυτό αντικαθιστά κάθε pixel με τη διάμεση τιμή
    των pixel στο τετραγωνικό παράθυρο k×k γύρω του.
    Έτσι, εξαλείφει αποτελεσματικά θόρυβο τύπου "salt & pepper"
    χωρίς να θολώνει τόσο πολύ τις ακμές όσο το mean filter.
    """

    # Ελέγχουμε ότι το k είναι περιττός αριθμός (ώστε να υπάρχει κέντρο)
    # και ότι είναι τουλάχιστον 3 (π.χ. 3x3, 5x5, 7x7 κ.λπ.).
    assert k % 2 == 1 and k >= 3

    # Επεκτείνουμε την εικόνα γύρω γύρω με αντανάκλαση (reflect padding),
    # ώστε το φίλτρο να μπορεί να εφαρμοστεί κανονικά και στα άκρα
    # χωρίς να χάνουμε πληροφορία ή να χρειάζεται ειδική μεταχείριση.
    pad = pad_reflect(img, k)

    # Υπολογίζουμε την "ακτίνα" r του παραθύρου:
    # Αν k = 3 → r = 1 (1 pixel γύρω από το κέντρο)
    # Αν k = 5 → r = 2, κ.ο.κ.
    r = k // 2

    # Δημιουργούμε κενή εικόνα εξόδου με τις ίδιες διαστάσεις όπως η αρχική.
    out = np.empty_like(img)

    # Παίρνουμε το ύψος (H) και το πλάτος (W) της αρχικής εικόνας.
    H, W = img.shape

    # Για κάθε pixel της αρχικής εικόνας:
    for y in range(H):
        for x in range(W):
            # Επιλέγουμε το παράθυρο k×k γύρω από το pixel (y, x)
            # πάνω στην "padded" εικόνα.
            #
            # π.χ. αν k=3 και (y,x)=(10,10) → παίρνουμε pixels [9:12, 9:12]
            # (δηλαδή το pixel, συν 1 γύρω του σε κάθε κατεύθυνση).
            win = pad[y:y + 2*r + 1, x:x + 2*r + 1]

            # Υπολογίζουμε τη διάμεση τιμή (median) των pixel στο παράθυρο.
            #
            # Η διάμεση είναι η "μεσαία" τιμή αν ταξινομήσουμε όλα τα pixel:
            #   - Αν υπάρχουν ακραίες τιμές (π.χ. 0 ή 255 λόγω θορύβου),
            #     αυτές δεν επηρεάζουν το αποτέλεσμα όσο στο μέσο όρο.
            out[y, x] = np.median(win)

    # Επιστρέφουμε τη φιλτραρισμένη εικόνα.
    # (Οι τιμές είναι ήδη έγκυρες, συνήθως uint8.)
    return out


Το median filter "κοιτάζει" κάθε pixel και τα γειτονικά του. Αν βρει ακραίες τιμές (πολύ σκοτεινά ή πολύ φωτεινά σημεία που δεν ταιριάζουν), τις αγνοεί και κρατά τη «μεσαία» τιμή. Έτσι καθαρίζει τον “salt & pepper” θόρυβο χωρίς να αλλοιώνει σημαντικά τα όρια (edges).

In [20]:
def integral_image(img: np.ndarray) -> np.ndarray:
    """
    Υπολογίζει την "αθροιστική εικόνα" (integral image) μιας γκρίζας εικόνας.

    Η integral image σε κάθε σημείο (x, y) περιέχει το άθροισμα όλων των pixel
    που βρίσκονται πάνω και αριστερά από το (x, y), συμπεριλαμβανομένου του ίδιου.

    Δηλαδή:
        I_int(x, y) = sum(img[0:y, 0:x])

    Με αυτή τη δομή μπορούμε να βρίσκουμε το άθροισμα οποιουδήποτε ορθογωνίου
    περιοχής με μόνο 4 αναφορές σε στοιχεία του πίνακα (O(1) χρόνος).

    Επιστρέφει:
    ----------
    np.ndarray:
        Την integral image με ένα επιπλέον "περιθώριο" μηδενικών γύρω γύρω
        (διαστάσεις (H+1, W+1)), ώστε η δεικτοδότηση να είναι πιο απλή.
    """

    # Μετατρέπουμε την εικόνα σε float64 για ακρίβεια στους υπολογισμούς.
    # Αν μέναμε σε uint8 (0–255), θα είχαμε υπερχειλίσεις σε μεγάλα αθροίσματα.
    img64 = img.astype(np.float64)

    # Υπολογίζουμε αθροιστικά τις τιμές pixel πρώτα κατά στήλες (axis=0)
    # και μετά κατά γραμμές (axis=1).
    #
    # Η μέθοδος .cumsum() δίνει το αθροιστικό άθροισμα:
    # π.χ. [1, 2, 3] → [1, 3, 6]
    #
    # Εφαρμόζοντας δύο φορές (κάθετα και οριζόντια) παίρνουμε το συνολικό άθροισμα
    # όλων των pixel πάνω και αριστερά από κάθε σημείο.
    integ = img64.cumsum(axis=0).cumsum(axis=1)

    # Για να κάνουμε ευκολότερη τη δεικτοδότηση ορθογωνίων περιοχών,
    # προσθέτουμε ένα "περιθώριο" από μηδενικά γύρω από την integral image.
    #
    # Έτσι, η τελική integral image έχει μέγεθος (H+1, W+1),
    # και το στοιχείο integ_padded[y+1, x+1] αντιστοιχεί στο άθροισμα
    # της περιοχής από (0,0) έως (y, x).
    integ_padded = np.zeros((integ.shape[0] + 1, integ.shape[1] + 1), dtype=np.float64)

    # Τοποθετούμε την "κανονική" integral image μέσα στην padded εκδοχή,
    # αφήνοντας το πρώτο row και column μηδενικά.
    integ_padded[1:, 1:] = integ

    # Επιστρέφουμε την padded integral image.
    return integ_padded


Η integral image λειτουργεί σαν “σωρευτικός χάρτης αθροισμάτων”. Αν τη φτιάξεις μία φορά, μπορείς να υπολογίζεις το άθροισμα των τιμών οποιασδήποτε περιοχής της εικόνας σε ένα βήμα αντί να αθροίζεις ξανά και ξανά όλα τα pixel μέσα στο παράθυρο.

In [21]:
def rect_sum(integ: np.ndarray, x0: int, y0: int, x1: int, y1: int) -> float:
    """
    Υπολογίζει το άθροισμα όλων των τιμών pixel μέσα σε ένα ορθογώνιο
    τμήμα της αρχικής εικόνας, χρησιμοποιώντας την integral image.

    Το ορθογώνιο καθορίζεται από τις γωνίες:
        (x0, y0) → πάνω-αριστερή
        (x1, y1) → κάτω-δεξιά

    Οι συντεταγμένες είναι inclusive, δηλαδή περιλαμβάνουν και το (x1, y1).

    Επιστρέφει:
    -----------
    float
        Το άθροισμα όλων των pixel μέσα στο ορθογώνιο.
    """

    # Επειδή η integral image που δημιουργήσαμε έχει ένα επιπλέον
    # "περιθώριο" (padding) μηδενικών γύρω-γύρω (δηλαδή είναι (H+1, W+1)),
    # μετακινούμε όλες τις συντεταγμένες κατά +1 για να ταιριάζουν.
    x0p, y0p, x1p, y1p = x0 + 1, y0 + 1, x1 + 1, y1 + 1

    # Ο υπολογισμός βασίζεται στο θεώρημα του αθροίσματος περιοχής:
    #
    # Αν έχουμε integral image I, τότε το άθροισμα των τιμών
    # στο ορθογώνιο που ορίζεται από (x0, y0) έως (x1, y1) είναι:
    #
    #   Sum = D + A - B - C
    #
    # όπου:
    #   A = I(y0, x0)        πάνω-αριστερή γωνία
    #   B = I(y0, x1+1)      πάνω-δεξιά
    #   C = I(y1+1, x0)      κάτω-αριστερή
    #   D = I(y1+1, x1+1)    κάτω-δεξιά
    #
    # Έτσι αποφεύγουμε τη διπλή καταμέτρηση και παίρνουμε το σωστό άθροισμα.

    A = integ[y0p - 1, x0p - 1]  # πάνω-αριστερά
    B = integ[y0p - 1, x1p]      # πάνω-δεξιά
    C = integ[y1p,     x0p - 1]  # κάτω-αριστερά
    D = integ[y1p,     x1p]      # κάτω-δεξιά

    # Επιστρέφουμε το άθροισμα των pixel στο ζητούμενο ορθογώνιο.
    return A + D - B - C


η συνάρτηση μπορεί να βρει το άθροισμα όλων των pixel μέσα σε οποιοδήποτε ορθογώνιο της εικόνας με 4 μόνο προσπελάσεις μνήμης, ανεξάρτητα από το μέγεθος του ορθογωνίου.

In [22]:
def mean_gray_in_box(integ: np.ndarray, x: int, y: int, w: int, h: int) -> float:
    """
    Υπολογίζει τη μέση τιμή (μέσο γκρι) όλων των pixel
    μέσα σε ένα ορθογώνιο τμήμα της εικόνας.

    Χρησιμοποιεί την integral image για να βρει το άθροισμα των pixel
    πολύ γρήγορα, και μετά διαιρεί με το εμβαδόν του παραθύρου.

    Παράμετροι
    ----------
    integ : np.ndarray
        Η integral image της αρχικής εικόνας.
    x, y : int
        Οι συντεταγμένες της πάνω-αριστερής γωνίας του ορθογωνίου.
    w, h : int
        Το πλάτος και το ύψος του ορθογωνίου σε pixel.

    Επιστρέφει
    ----------
    float
        Τη μέση φωτεινότητα των pixel μέσα στο ορθογώνιο.
    """

    # Υπολογίζουμε το άθροισμα των pixel μέσα στο ορθογώνιο:
    # - Η πάνω-αριστερή γωνία είναι στο (x, y)
    # - Η κάτω-δεξιά γωνία είναι στο (x + w - 1, y + h - 1)
    #
    # Η rect_sum() χρησιμοποιεί την integral image και επιστρέφει
    # το συνολικό άθροισμα των τιμών pixel στην περιοχή αυτή.
    s = rect_sum(integ, x, y, x + w - 1, y + h - 1)

    # Ο μέσος όρος είναι απλώς το συνολικό άθροισμα
    # διαιρεμένο με το πλήθος των pixel της περιοχής (πλάτος × ύψος).
    return s / (w * h)


Αυτή η συνάρτηση λέει στο πρόγραμμα:

«Πες μου πόσο φωτεινή είναι κατά μέσο όρο μια συγκεκριμένη περιοχή της εικόνας.»

Χρησιμοποιεί την integral image για να το κάνει πολύ γρήγορα, χωρίς να χρειάζεται να προσθέτει όλα τα pixel ένα-ένα

In [23]:
def binarize_otsu(img_gray: np.ndarray, invert: bool = True) -> Tuple[float, np.ndarray]:
    """
    Εκτελεί δυαδικοποίηση (binarization) μιας γκρίζας εικόνας
    χρησιμοποιώντας τον αλγόριθμο Otsu.

    Η μέθοδος αυτή υπολογίζει αυτόματα το κατώφλι φωτεινότητας (threshold)
    που διαχωρίζει τη "φόντο" περιοχή από το "αντικείμενο" της εικόνας.

    Παράμετροι
    ----------
    img_gray : np.ndarray
        Η εικόνα σε γκρίζα κλίμακα (τιμές 0–255).
    invert : bool, προεπιλογή True
        Αν είναι True, η δυαδική εικόνα αντιστρέφεται (τα φωτεινά γίνονται μαύρα).
        Αν είναι False, αφήνει τα φωτεινά ως λευκά και τα σκοτεινά ως μαύρα.

    Επιστρέφει
    ----------
    Tuple[float, np.ndarray]
        - thr: Το κατώφλι (threshold) που βρήκε αυτόματα ο Otsu.
        - th : Η δυαδική (ασπρόμαυρη) εικόνα που προκύπτει.
    """

    # Επιλέγουμε το flag που δηλώνει ότι θα χρησιμοποιηθεί η μέθοδος Otsu.
    # Το cv2.THRESH_OTSU ζητά από το OpenCV να υπολογίσει αυτόματα το threshold.
    flag = cv2.THRESH_OTSU

    # Αν ζητήσουμε "αντιστροφή" (invert=True), προσθέτουμε και το flag
    # cv2.THRESH_BINARY_INV, που κάνει το εξής:
    #   - Pixel φωτεινότερα από το κατώφλι → 0 (μαύρα)
    #   - Pixel σκοτεινότερα → 255 (λευκά)
    #
    # Αν δεν το βάλουμε, η δυαδικοποίηση γίνεται κανονικά:
    #   - Pixel φωτεινότερα → λευκά
    #   - Pixel σκοτεινότερα → μαύρα
    if invert:
        flag |= cv2.THRESH_BINARY_INV

    # Η συνάρτηση cv2.threshold εκτελεί τη δυαδικοποίηση:
    # Παράμετροι:
    #   - img_gray: η είσοδος (γκρίζα εικόνα)
    #   - 0: placeholder τιμή (αγνοείται, γιατί ο Otsu υπολογίζει μόνος του το κατώφλι)
    #   - 255: μέγιστη τιμή για τα λευκά pixel
    #   - flag: καθορίζει τον τύπο κατωφλίου (εδώ Otsu + binary_inv ή όχι)
    #
    # Επιστρέφει δύο τιμές:
    #   - thr: το κατώφλι που υπολόγισε ο Otsu
    #   - th : η δυαδική εικόνα (0 = μαύρο, 255 = λευκό)
    thr, th = cv2.threshold(img_gray, 0, 255, flag)

    # Επιστρέφουμε και το κατώφλι και την τελική εικόνα.
    return thr, th


Η συνάρτηση αυτή μετατρέπει μια εικόνα σε ασπρόμαυρη χωρίς να χρειάζεται εμείς να ορίσουμε κατώφλι. Ο Otsu αναλύει το ιστόγραμμα της εικόνας και επιλέγει αυτόματα τη φωτεινότητα που χωρίζει βέλτιστα το φόντο από τα αντικείμενα.

In [24]:
def text_region_masks(th_bin: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Δημιουργεί δύο "μάσκες" (δηλαδή εικόνες που δείχνουν τις περιοχές ενδιαφέροντος)
    από μια ήδη δυαδική εικόνα (μετά τη binarization).

    - texts_bound:  μάσκα που περιλαμβάνει μεγαλύτερες περιοχές κειμένου (π.χ. παραγράφους)
                    χρησιμοποιώντας διαστολή (dilation) και κλείσιμο (closing).
    - words_bound:  μάσκα που περιλαμβάνει μεμονωμένες λέξεις,
                    μέσω οριζόντιας σύνδεσης χαρακτήρων.

    Επιστρέφει
    ----------
    Tuple[np.ndarray, np.ndarray]
        texts_bound, words_bound
    """

    # Δημιουργούμε "δομικά στοιχεία" (structuring elements),
    # δηλαδή μικρούς πίνακες 1/0 που καθορίζουν το σχήμα και το μέγεθος
    # των μορφολογικών πράξεων (όπως η διαστολή ή το κλείσιμο).

    # Μικρό τετράγωνο 3×3 για αρχική διαστολή (ενώνει κοντινά pixel)
    sqr_1 = np.ones((3, 3), np.uint8)

    # Μεγαλύτερο τετράγωνο 5×5 για ισχυρότερο "κλείσιμο" (ένωση ευρύτερων περιοχών)
    sqr_2 = np.ones((5, 5), np.uint8)

    # Ορθογώνιο 1×4, δηλαδή ένα οριζόντιο παραλληλόγραμμο.
    # Θα χρησιμοποιηθεί για να ενώσει γράμματα σε λέξεις, επειδή τα γράμματα
    # συνήθως είναι τοποθετημένα οριζόντια.
    rect_1 = np.ones((1, 4), np.uint8)

    # === Δημιουργία μάσκας texts_bound (για ολόκληρες περιοχές κειμένου) ===

    # 1️⃣ Πρώτα κάνουμε μια διαστολή (dilation) με το μικρό τετράγωνο.
    # Η διαστολή μεγαλώνει τα λευκά pixel, με αποτέλεσμα να ενώνει μικρά κενά
    # ανάμεσα σε γράμματα ή γραμμές.
    tmp = cv2.morphologyEx(th_bin, cv2.MORPH_DILATE, sqr_1, iterations=1)

    # 2️⃣ Μετά εφαρμόζουμε "κλείσιμο" (closing = dilation ακολουθούμενο από erosion)
    # με μεγαλύτερο τετράγωνο στοιχείο, αρκετές φορές (8 επαναλήψεις).
    # Το closing γεμίζει μικρά κενά και ενώνει ομάδες γραμμάτων ή λέξεων
    # σε μεγαλύτερες περιοχές (π.χ. παραγράφους).
    texts_bound = cv2.morphologyEx(tmp, cv2.MORPH_CLOSE, sqr_2, iterations=8)

    # === Δημιουργία μάσκας words_bound (για μεμονωμένες λέξεις) ===

    # 3️⃣ Εδώ χρησιμοποιούμε το οριζόντιο στοιχείο (rect_1) για διαστολή,
    # ώστε να ενωθούν μόνο τα γράμματα μιας ίδιας λέξης (όχι διαφορετικές γραμμές).
    tmp2 = cv2.morphologyEx(th_bin, cv2.MORPH_DILATE, rect_1, iterations=3)

    # 4️⃣ Έπειτα εφαρμόζουμε "κλείσιμο" ξανά, για να σταθεροποιηθούν τα όρια των λέξεων.
    words_bound = cv2.morphologyEx(tmp2, cv2.MORPH_CLOSE, rect_1, iterations=1)

    # Επιστρέφουμε και τις δύο μάσκες.
    return texts_bound, words_bound


Αυτή η συνάρτηση παίρνει μια ασπρόμαυρη εικόνα (όπου το κείμενο είναι λευκό πάνω σε μαύρο φόντο) και δημιουργεί δύο “μάσκες”:

Μία που δείχνει ολόκληρες περιοχές κειμένου (παράγραφοι ή μπλοκ),

Και μία που δείχνει μεμονωμένες λέξεις.

Χρησιμοποιεί “μορφολογικές πράξεις” όπως διαστολή (dilation) και κλείσιμο (closing), οι οποίες βοηθούν να «κολλήσουν» μεταξύ τους γράμματα που ανήκουν στην ίδια περιοχή ή λέξη.

Έχεις δύο στόχους:

Υποπεριοχές κειμένου (π.χ. παραγράφους/γραμμές).

Ξεχωριστές λέξεις.

Για αυτό:

sqr_1 (3×3) και sqr_2 (5×5) είναι τετραγωνικά structuring elements.

rect_1 (1×4) είναι οριζόντιο στοιχείο, κατάλληλο για σύνδεση χαρακτήρων σε λέξη.

Ροή για texts_bound:

dilate με 3×3: μεγαλώνει λίγο το κείμενο, γεφυρώνει μικρά κενά.

closing με 5×5, iterations=8: closing = dilation + erosion.

Με πολλές επαναλήψεις, “γεφυρώνει” κενά μέσα στην ίδια γραμμή/μπλοκ κειμένου.

Έτσι, χαρακτήρες/λέξεις που ανήκουν στην ίδια “υποπεριοχή” θα ενωθούν σε μια μεγαλύτερη συνιστώσα.

Ροή για words_bound:

dilate με (1×4) οριζόντιο:

Συνδέει χαρακτήρες κατά μήκος της γραμμής, σχηματίζοντας λέξεις.

closing με (1×4):

Κλείνει μικρά κενά μέσα στη λέξη.

Με αυτόν τον τρόπο:

texts_bound → για εντοπισμό γενικών περιοχών κειμένου.

words_bound → για μέτρηση λέξεων μέσα στο κάθε bbox.

In [25]:
def horizontal_projection(th_bin: np.ndarray) -> np.ndarray:
    """
    Υπολογίζει την *οριζόντια προβολή* (horizontal projection) μιας δυαδικής εικόνας.

    Δηλαδή, για κάθε γραμμή (row) της εικόνας μετρά πόσα "λευκά" pixel (foreground)
    υπάρχουν. Το αποτέλεσμα είναι ένας πίνακας όπου κάθε στοιχείο αντιστοιχεί
    στην πυκνότητα του κειμένου σε εκείνη τη γραμμή.

    Πολύ χρήσιμο για:
    - εντοπισμό γραμμών κειμένου (text line segmentation)
    - ανάλυση διάταξης σε έγγραφα (layout analysis)
    """

    # Υποθέτουμε ότι τα pixel του "foreground" (δηλαδή γράμματα/κείμενο)
    # έχουν τιμή 255, ενώ το φόντο είναι 0.
    #
    # Με το (th_bin // 255) μετατρέπουμε:
    #   - 255 → 1
    #   - 0   → 0
    #
    # Έτσι μπορούμε να μετρήσουμε εύκολα το πλήθος των λευκών pixel.
    #
    # Το .sum(axis=1) προσθέτει όλα τα pixel κάθε γραμμής (κατά μήκος των στηλών).
    # Το αποτέλεσμα είναι ένας μονοδιάστατος πίνακας: ένα νούμερο για κάθε γραμμή.
    return (th_bin // 255).sum(axis=1)


def vertical_projection(th_bin: np.ndarray) -> np.ndarray:
    """
    Υπολογίζει την *κατακόρυφη προβολή* (vertical projection) μιας δυαδικής εικόνας.

    Δηλαδή, για κάθε στήλη (column) της εικόνας μετρά πόσα "λευκά" pixel (foreground)
    υπάρχουν. Το αποτέλεσμα δείχνει πού υπάρχουν γράμματα ή κάθετες περιοχές κειμένου.

    Πολύ χρήσιμο για:
    - ανίχνευση ορίων λέξεων (word segmentation)
    - αναγνώριση χαρακτήρων (OCR preprocessing)
    """

    # Μετατροπή των pixel του foreground (255) σε 1, όπως και πριν,
    # ώστε να μπορούμε να μετρήσουμε εύκολα τον αριθμό των "ενεργών" pixel ανά στήλη.
    #
    # Το .sum(axis=0) προσθέτει τα pixel κάθε στήλης (κατά μήκος των γραμμών).
    # Επιστρέφει έναν πίνακα με μήκος όσο το πλάτος της εικόνας.
    return (th_bin // 255).sum(axis=0)


Η horizontal_projection σου δείχνει πόσο “γεμάτη” είναι κάθε γραμμή με γράμματα. Έτσι μπορείς να βρεις τα όρια μεταξύ σειρών κειμένου.

Η vertical_projection σου δείχνει πόσο “γεμάτη” είναι κάθε στήλη, άρα μπορείς να εντοπίσεις διαχωριστικά ανάμεσα σε λέξεις ή χαρακτήρες.

In [26]:
def regions_from_horizontal_projection(th_bin: np.ndarray, min_run: int = 5) -> List[Tuple[int, int]]:
    """
    Εντοπίζει περιοχές (γραμμές κειμένου) σε μια δυαδική εικόνα
    χρησιμοποιώντας την οριζόντια προβολή (horizontal projection).

    Η ιδέα:
    - Υπολογίζουμε πόσα "λευκά" pixel (γράμματα) υπάρχουν σε κάθε γραμμή της εικόνας.
    - Οι περιοχές όπου το πλήθος είναι > 0 αντιστοιχούν σε κείμενο.
    - Οι γραμμές όπου το πλήθος = 0 είναι κενό (διάστημα μεταξύ γραμμών).

    Παράμετροι
    ----------
    th_bin : np.ndarray
        Η δυαδική (ασπρόμαυρη) εικόνα μετά τη binarization.
    min_run : int, προεπιλογή 5
        Ελάχιστο ύψος περιοχής (σε γραμμές pixel) για να θεωρηθεί έγκυρη
        γραμμή κειμένου. Εξαλείφει μικρούς τυχαίους "θορύβους".

    Επιστρέφει
    ----------
    List[Tuple[int, int]]
        Λίστα από ζεύγη (y0, y1), όπου κάθε ζεύγος δηλώνει
        το πάνω και κάτω όριο μιας γραμμής κειμένου.
    """

    # Υπολογίζουμε την οριζόντια προβολή:
    # ένα διάνυσμα που δείχνει πόσα "ενεργά" pixel υπάρχουν σε κάθε γραμμή.
    hp = horizontal_projection(th_bin)

    # Θα αποθηκεύσουμε εδώ τις περιοχές που βρέθηκαν.
    runs = []

    # Σημαία για το αν βρισκόμαστε μέσα σε "τρέχουσα περιοχή κειμένου".
    in_run = False

    # Μεταβλητή για το πού ξεκινά η τρέχουσα περιοχή (αν υπάρχει).
    start = 0

    # Διατρέχουμε κάθε γραμμή (y) της εικόνας και την αντίστοιχη τιμή v
    # από την προβολή (δηλαδή πόσα λευκά pixel έχει).
    for y, v in enumerate(hp):

        # Αν βρούμε γραμμή με περιεχόμενο (v > 0) και δεν ήμασταν ήδη σε περιοχή κειμένου:
        # ξεκινάμε νέα "τρέχουσα περιοχή".
        if v > 0 and not in_run:
            in_run = True
            start = y

        # Αν βρούμε γραμμή χωρίς περιεχόμενο (v == 0) ενώ ήμασταν σε περιοχή κειμένου:
        # σημαίνει ότι η προηγούμενη περιοχή τελείωσε.
        elif v == 0 and in_run:
            # Ελέγχουμε αν η περιοχή είναι αρκετά "ψηλή" (>= min_run)
            # ώστε να είναι πραγματική γραμμή και όχι μικρός θόρυβος.
            if y - start >= min_run:
                runs.append((start, y - 1))
            # Κλείνουμε την τρέχουσα περιοχή.
            in_run = False

    # Αν η εικόνα τελειώσει ενώ είμαστε ακόμα "μέσα" σε περιοχή (χωρίς να έχει βρεθεί κενό),
    # τότε προσθέτουμε και αυτή την τελευταία περιοχή.
    if in_run and (len(hp) - start) >= min_run:
        runs.append((start, len(hp) - 1))

    # Επιστρέφουμε λίστα από (y0, y1),
    # δηλαδή τα πάνω–κάτω όρια κάθε γραμμής κειμένου.
    return runs  # λίστα από (y0, y1)


Η συνάρτηση “διαβάζει” την εικόνα οριζόντια, από πάνω προς τα κάτω, και ψάχνει για ομάδες γραμμών που έχουν “μελάνι” (δηλαδή λευκά pixel πάνω σε μαύρο φόντο).
Όταν βρει τέτοια ομάδα, τη θεωρεί γραμμή κειμένου και επιστρέφει τα όριά της (πάνω και κάτω).

hp[y] = πόσα pixels κειμένου έχει η γραμμή y.

in_run = αν αυτή τη στιγμή είσαι σε “ζώνη κειμένου”.

Όταν:

v > 0 και δεν είσαι σε run → ξεκινάς νέο run (start = y).

v == 0 και είσαι σε run → κλείνεις το run (τέλος = y-1).

Αν το μήκος (y - start) είναι πιο μικρό από min_run, το πετάς ως πολύ μικρό/θόρυβο.

Στο τέλος, αν το run συνεχίζει μέχρι το τέλος του πίνακα, το προσθέτεις.

In [27]:
def count_words_by_vproj(th_region: np.ndarray, zero_gap_thresh: int = 3) -> int:
    """
    Μετρά πόσες "λέξεις" υπάρχουν μέσα σε μια περιοχή κειμένου
    χρησιμοποιώντας την κατακόρυφη προβολή (vertical projection).

    Η λογική:
    - Υπολογίζουμε πόσα pixel κειμένου υπάρχουν σε κάθε στήλη.
    - Οι στήλες με τιμές > 0 δείχνουν ότι υπάρχει γράμμα.
    - Οι στήλες με 0 είναι κενά.
    - Αν ένα κενό είναι αρκετά μεγάλο (>= zero_gap_thresh),
      θεωρούμε ότι χωρίζει δύο διαφορετικές λέξεις.

    Παράμετροι
    ----------
    th_region : np.ndarray
        Δυαδική εικόνα (0 = φόντο, 255 = γράμματα) που περιέχει μια μόνο γραμμή κειμένου.
    zero_gap_thresh : int, προεπιλογή 3
        Ο ελάχιστος αριθμός συνεχόμενων "κενών" στηλών που θεωρείται διαχωριστικό λέξεων.

    Επιστρέφει
    ----------
    int
        Τον αριθμό των λέξεων που εντοπίστηκαν.
    """

    # Υπολογίζουμε την κατακόρυφη προβολή:
    # Κάθε στοιχείο δείχνει πόσα "ενεργά" (λευκά) pixel υπάρχουν στη συγκεκριμένη στήλη.
    vp = vertical_projection(th_region)

    # Αρχικοποιούμε μετρητές:
    words = 0      # πλήθος λέξεων
    gap = 0        # μέγεθος τρέχοντος κενού (σε στήλες)
    in_word = False  # αν αυτή τη στιγμή βρισκόμαστε "μέσα" σε λέξη

    # Διατρέχουμε κάθε τιμή της κατακόρυφης προβολής
    # (μία τιμή για κάθε στήλη της εικόνας)
    for v in vp:

        # Αν η στήλη έχει περιεχόμενο (pixel > 0):
        if v > 0:
            # Αν δεν ήμασταν ήδη μέσα σε λέξη, σημαίνει ότι ξεκινάμε καινούρια λέξη
            if not in_word:
                in_word = True
                words += 1  # Μετράμε τη νέα λέξη
            # Μηδενίζουμε το μέτρημα των "κενών" στηλών
            gap = 0

        # Αν η στήλη είναι κενή (v == 0):
        else:
            # Αν ήμασταν μέσα σε λέξη, αυξάνουμε το μέγεθος του κενού
            if in_word:
                gap += 1
                # Αν το κενό γίνει αρκετά μεγάλο (>= zero_gap_thresh),
                # θεωρούμε ότι η λέξη τελείωσε.
                if gap >= zero_gap_thresh:
                    in_word = False
                    gap = 0

    # Επιστρέφουμε τον αριθμό των λέξεων που μετρήθηκαν
    return words


Η συνάρτηση «κοιτάει» τη γραμμή της εικόνας κατά πλάτος (οριζόντια) και μετρά πότε ξεκινά και τελειώνει μια ομάδα από στήλες που περιέχουν γράμματα.
Κάθε φορά που συναντά ένα κενό αρκετά μεγάλο, καταλαβαίνει ότι τελείωσε μία λέξη και η επόμενη ομάδα pixel ανήκει σε καινούρια λέξη.

In [28]:
def analyze_regions(img_gray: np.ndarray, th_bin: np.ndarray, texts_bound: np.ndarray, words_bound: np.ndarray,
                    out_dir: str, tag: str) -> None:
    """
    Αναλύει τις περιοχές κειμένου σε μια εικόνα και υπολογίζει βασικά μετρικά για καθεμία:
    - εμβαδό bounding box
    - εμβαδό κειμένου μέσα στο box (σε pixel)
    - αριθμό λέξεων
    - μέση γκρι τιμή στην περιοχή

    Επιπλέον:
    - σχεδιάζει τα bounding boxes πάνω στην αρχική εικόνα
    - αποθηκεύει τα αποτελέσματα σε αρχείο εικόνας (.png) και σε αρχείο κειμένου (.txt)
    """

    # Εντοπίζουμε τα συνδεδεμένα συστατικά (connected components) στη μάσκα texts_bound,
    # δηλαδή "μπλοκ κειμένου". Η texts_bound είναι δυαδική εικόνα όπου οι περιοχές
    # κειμένου είναι 255 και το φόντο 0.
    #
    # num   : πόσα labels (συστατικά) βρέθηκαν, συμπεριλαμβανομένου του φόντου
    # labels: εικόνα ίδιου μεγέθους, όπου κάθε pixel έχει τιμή το id της περιοχής στην οποία ανήκει
    num, labels = cv2.connectedComponents(texts_bound)

    # Μετατρέπουμε τη γκρίζα εικόνα σε BGR (3 κανάλια), ώστε να μπορούμε να σχεδιάζουμε
    # έγχρωρα ορθογώνια (bounding boxes) πάνω της.
    color = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

    # Υπολογίζουμε την integral image της γκρίζας εικόνας, για να μπορούμε μετά
    # να βρίσκουμε γρήγορα τη μέση γκρι τιμή σε κάθε ορθογώνια περιοχή.
    integ = integral_image(img_gray)

    # Μετρητής μικρών περιοχών που απορρίπτονται (π.χ. θόρυβος)
    counter_small = 0

    # Λίστα όπου θα αποθηκεύσουμε τα αποτελέσματα (ένα tuple ανά περιοχή).
    # Κάθε στοιχείο θα είναι: (region_id, bbox_area, text_area, n_words, mean_gray)
    results = []

    # Ξεκινάμε από 1, γιατί το label 0 αντιστοιχεί στο φόντο.
    for i in range(1, num):
        # Δημιουργούμε μια μάσκα (binary image) για την τρέχουσα περιοχή με label i.
        # Όπου labels == i → 255 (foreground), αλλιώς 0.
        mask = (labels == i).astype(np.uint8) * 255

        # Υπολογίζουμε το bounding rectangle γύρω από τη συγκεκριμένη περιοχή.
        # Επιστρέφει:
        #   x, y : πάνω-αριστερή γωνία
        #   w, h : πλάτος και ύψος του ορθογωνίου
        x, y, w, h = cv2.boundingRect(mask)

        # Φιλτράρουμε πολύ μικρές περιοχές (π.χ. θόρυβος, κουκκίδες) με βάση
        # ελάχιστο πλάτος/ύψος. Αν είναι πολύ μικρή, δεν την μετράμε ως περιοχή κειμένου.
        if w < 12 or h < 12:
            counter_small += 1
            continue

        # Δίνουμε ένα συνεχόμενο id στις "έγκυρες" περιοχές, αφαιρώντας όσες
        # έχουν απορριφθεί μέχρι τώρα ως μικρές.
        region_id = i - counter_small

        # --- Οπτικοποίηση: σχεδίαση πλαισίων και id πάνω στην εικόνα ---

        # Σχεδιάζουμε το bounding box με κόκκινο χρώμα (0,0,255) και πάχος 2 pixel.
        cv2.rectangle(color, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # Γράφουμε το id της περιοχής (region_id) λίγο πάνω από το πλαίσιο.
        cv2.putText(color, str(region_id), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 0, 255), 2)

        # --- Υπολογισμός μετρικών για την περιοχή ---

        # Συνολικό εμβαδό του bounding box σε pixel.
        bbox_area = w * h

        # Εμβαδό "πραγματικού" κειμένου μέσα στο box:
        # Κόβουμε από τη δυαδική εικόνα th_bin το αντίστοιχο ορθογώνιο [y:y+h, x:x+w]
        text_crop = th_bin[y:y + h, x:x + w]

        # Με το (text_crop // 255) μετατρέπουμε τα 255 → 1 και 0 → 0,
        # οπότε το άθροισμα δίνει πόσα pixel κειμένου υπάρχουν.
        text_area = int((text_crop // 255).sum())

        # Υπολογισμός αριθμού λέξεων στην ίδια περιοχή:
        # Κόβουμε τη μάσκα words_bound στο ίδιο ορθογώνιο.
        words_crop = words_bound[y:y + h, x:x + w]

        # Εντοπίζουμε συνδεδεμένα συστατικά στη μάσκα λέξεων.
        # Κάθε συστατικό εδώ αντιστοιχεί (περίπου) σε μία λέξη.
        n_words, _ = cv2.connectedComponents(words_crop)

        # Αφαιρούμε 1 για το φόντο (label 0). Δεν μπορεί να είναι αρνητικό.
        n_words = max(n_words - 1, 0)

        # Υπολογισμός μέσης γκρι τιμής μέσα στο bounding box
        # χρησιμοποιώντας την integral image και τη helper συνάρτηση mean_gray_in_box().
        mean_gray = mean_gray_in_box(integ, x, y, w, h)

        # Αποθηκεύουμε τα αποτελέσματα για την περιοχή στη λίστα.
        results.append((region_id, bbox_area, text_area, n_words, mean_gray))

    # Αποθήκευση της εικόνας με τα bounding boxes στο δίσκο.
    # Το filename περιλαμβάνει το tag, ώστε να ξεχωρίζουν διαφορετικά runs.
    save_img(os.path.join(out_dir, f"bounding_boxes_{tag}.png"), color)

    # Αποθήκευση των μετρικών σε αρχείο κειμένου (π.χ. για αναφορά/στατιστική ανάλυση).
    with open(os.path.join(out_dir, f"metrics_{tag}.txt"), "w", encoding="utf-8") as f:
        for r in results:
            f.write(f"------- Region {r[0]} -------\n")
            f.write(f"Bounding box area (px): {r[1]}\n")
            f.write(f"Text area (px): {r[2]}\n")
            f.write(f"Words: {r[3]}\n")
            f.write(f"Mean gray-level in bbox: {r[4]}\n\n")


Για κάθε μπλοκ κειμένου που βρίσκει:

Το εντοπίζει ως connected component στη μάσκα texts_bound.

Υπολογίζει το bounding box και φιλτράρει πολύ μικρές περιοχές.

Βγάζει:

εμβαδό box,

πόσα pixel κειμένου έχει μέσα,

πόσες λέξεις υπάρχουν (από words_bound),

τη μέση γκρι τιμή της περιοχής (με integral image).

Σχεδιάζει τα πλαίσια στην εικόνα και

Γράφει όλα τα μετρικά σε .txt

In [29]:
def analyze_with_projection_profiles(img_gray: np.ndarray, th_bin: np.ndarray, out_dir: str, tag: str) -> None:
    """
    Αναλύει περιοχές κειμένου χρησιμοποιώντας προβολικά προφίλ (horizontal & vertical projections).

    Βήματα:
    1. Βρίσκει οριζόντιες "λωρίδες" κειμένου (γραμμές/υποπεριοχές) από την οριζόντια προβολή.
    2. Για κάθε τέτοια λωρίδα:
       - Υπολογίζει εμβαδό bounding box.
       - Υπολογίζει πόσο κείμενο (σε pixel) περιέχει.
       - Εκτιμά πόσες λέξεις έχει, από την κατακόρυφη προβολή (v-projection).
       - Υπολογίζει τη μέση γκρι τιμή στην περιοχή (με integral image).
    3. Αποθηκεύει:
       - Εικόνα με τις περιοχές σημειωμένες.
       - Αρχείο .txt με τα μετρικά ανά περιοχή.
    """

    # 1️⃣ Εντοπισμός οριζόντιων υποπεριοχών (π.χ. γραμμές κειμένου)
    # Η regions_from_horizontal_projection() χρησιμοποιεί την horizontal_projection()
    # για να βρει συνεχόμενες ομάδες γραμμών που περιέχουν κείμενο.
    # Επιστρέφει λίστα από (y0, y1), πάνω–κάτω όρια κάθε περιοχής.
    runs = regions_from_horizontal_projection(th_bin, min_run=5)

    # Δημιουργούμε έγχρωμη εκδοχή της γκρίζας εικόνας,
    # ώστε να σχεδιάζουμε πάνω της με χρώματα (π.χ. μπλε πλαίσια).
    color = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

    # Υπολογίζουμε την integral image της γκρίζας εικόνας, για γρήγορο υπολογισμό
    # της μέσης γκρι τιμής σε ορθογώνιες περιοχές (mean_gray_in_box).
    integ = integral_image(img_gray)

    # Ανοίγουμε αρχείο κειμένου όπου θα γράψουμε τα μετρικά
    # για κάθε προβολική περιοχή (P1, P2, ...).
    metrics_path = os.path.join(out_dir, f"metrics_projection_{tag}.txt")
    with open(metrics_path, "w", encoding="utf-8") as f:

        # Διατρέχουμε όλες τις περιοχές που βρήκαμε από την οριζόντια προβολή.
        # Η enumerate(runs, start=1) μας δίνει:
        #   idx : 1, 2, 3, ... (id περιοχής)
        #   (y0, y1) : όρια της περιοχής σε άξονα ύψους (γραμμές)
        for idx, (y0, y1) in enumerate(runs, start=1):

            # Για απλότητα, παίρνουμε όλο το πλάτος της εικόνας ως x-όρια,
            # από την αριστερή άκρη (0) μέχρι τη δεξιά (W-1).
            x0, x1 = 0, th_bin.shape[1] - 1

            # Υπολογίζουμε ύψος και πλάτος του bounding box.
            h = y1 - y0 + 1
            w = x1 - x0 + 1

            # Σχεδιάζουμε ένα μπλε (255,0,0) ορθογώνιο γύρω από την περιοχή
            # πάνω στην έγχρωμη εικόνα, για οπτικοποίηση.
            cv2.rectangle(color, (x0, y0), (x1, y1), (255, 0, 0), 2)

            # Γράφουμε το id της περιοχής (π.χ. P1, P2, ...) λίγο πάνω από το box.
            cv2.putText(color, f"P{idx}", (x0, y0 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

            # -------------------- ΜΕΤΡΙΚΑ ΠΕΡΙΟΧΗΣ --------------------

            # 1. Εμβαδό bounding box (σε pixel)
            bbox_area = w * h

            # 2. Εμβαδό κειμένου:
            # Κόβουμε από τη δυαδική εικόνα το ορθογώνιο [y0:y1+1, x0:x1+1].
            # Με (// 255) μετατρέπουμε 255 → 1 και 0 → 0, το άθροισμα δίνει
            # πόσα pixel κειμένου υπάρχουν στην περιοχή.
            text_region = th_bin[y0:y1+1, x0:x1+1]
            text_area = int((text_region // 255).sum())

            # 3. Εκτίμηση αριθμού λέξεων με κατακόρυφη προβολή (vertical projection).
            # Χρησιμοποιούμε την count_words_by_vproj() πάνω στην ίδια περιοχή.
            words_est = count_words_by_vproj(text_region, zero_gap_thresh=3)

            # 4. Μέση γκρι τιμή στο bounding box,
            #    υπολογισμένη από την integral image.
            mean_gray = mean_gray_in_box(integ, x0, y0, w, h)

            # Γράφουμε τα αποτελέσματα της περιοχής στο αρχείο κειμένου.
            f.write(f"------- Region P{idx} -------\n")
            f.write(f"Bounding box area (px): {bbox_area}\n")
            f.write(f"Text area (px): {text_area}\n")
            f.write(f"Words (v-proj est.): {words_est}\n")
            f.write(f"Mean gray-level in bbox: {mean_gray}\n\n")

    # Τέλος, αποθηκεύουμε την εικόνα με τα σημειωμένα projection regions
    # (τα μπλε ορθογώνια και τα labels P1, P2, ...).
    out_img_path = os.path.join(out_dir, f"projection_regions_{tag}.png")
    save_img(out_img_path, color)


Χωρίζει την εικόνα σε οριζόντιες λωρίδες κειμένου (με horizontal projection).

Για κάθε λωρίδα:

Μετρά πόσο χώρο καταλαμβάνει (bounding box area).

Μετρά πόσο από αυτόν τον χώρο είναι πραγματικό κείμενο (text area).

Εκτιμά πόσες λέξεις έχει, κοιτώντας τα κενά ανάμεσα σε ομάδες pixel (vertical projection).

Υπολογίζει τη μέση φωτεινότητα της περιοχής (mean gray).

Και στο τέλος:

Σώζει μια εικόνα με τα μπλε πλαίσια P1, P2, κ.λπ.

# **Μεθόδου 1: connected components πάνω στις μορφολογικές μάσκες.**

# Μεθόδου 2: projection profiles (row- & column-wise).

In [30]:
def process_document(image_path: str, out_dir: str = "outputs", k_mean: int = 3, k_median: int = 3) -> None:
    os.makedirs(out_dir, exist_ok=True)
    base = os.path.splitext(os.path.basename(image_path))[0]

    # 1) Ανάγνωση πρωτότυπης
    img = read_gray(image_path)
    save_img(os.path.join(out_dir, f"{base}_gray.png"), img)

    # 2) Δημιουργία δύο θορυβωδών αντιγράφων
    img_gn = add_gaussian_noise(img, sigma=15.0)
    img_sp = add_salt_pepper(img, amount=0.01, s_vs_p=0.5)
    save_img(os.path.join(out_dir, f"{base}_gaussian_noise.png"), img_gn)
    save_img(os.path.join(out_dir, f"{base}_saltpepper_noise.png"), img_sp)

    # 3) Αποθορυβοποίηση ΧΩΡΙΣ OpenCV filtering
    #    - Για Gaussian θόρυβο -> μέσο φίλτρο
    #    - Για S&P θόρυβο -> median φίλτρο
    den_gn = mean_filter(img_gn, k=k_mean)
    den_sp = median_filter(img_sp, k=k_median)
    save_img(os.path.join(out_dir, f"{base}_denoise_gaussian_mean{k_mean}.png"), den_gn)
    save_img(os.path.join(out_dir, f"{base}_denoise_saltpepper_median{k_median}.png"), den_sp)

    # 4) Δυαδικοποίηση Otsu
    thr0, th0 = binarize_otsu(img, invert=True)
    thr_gn, th_gn = binarize_otsu(den_gn, invert=True)
    thr_sp, th_sp = binarize_otsu(den_sp, invert=True)
    save_img(os.path.join(out_dir, f"{base}_bin_otsu_original.png"), th0)
    save_img(os.path.join(out_dir, f"{base}_bin_otsu_gaussian.png"), th_gn)
    save_img(os.path.join(out_dir, f"{base}_bin_otsu_saltpepper.png"), th_sp)

    # 5) Μάσκες υποπεριοχών & λέξεων
    texts0, words0 = text_region_masks(th0)
    texts_gn, words_gn = text_region_masks(th_gn)
    texts_sp, words_sp = text_region_masks(th_sp)
    save_img(os.path.join(out_dir, f"{base}_texts_mask_original.png"), texts0)
    save_img(os.path.join(out_dir, f"{base}_words_mask_original.png"), words0)

    # 6) Μετρήσεις (βασική μέθοδος με connected components)
    analyze_regions(img, th0, texts0, words0, out_dir, tag="original")
    analyze_regions(img, th_gn, texts_gn, words_gn, out_dir, tag="gaussian")
    analyze_regions(img, th_sp, texts_sp, words_sp, out_dir, tag="saltpepper")

    # 7) 2η μέθοδος: Projection Profiles (οριζόντιες περιοχές + κατακόρυφη εκτίμηση λέξεων)
    analyze_with_projection_profiles(img, th0, out_dir, tag="original")
    analyze_with_projection_profiles(img, th_gn, out_dir, tag="gaussian")
    analyze_with_projection_profiles(img, th_sp, out_dir, tag="saltpepper")

    # 8) Έτοιμο! Τα αποτελέσματα/εικόνες είναι στον φάκελο out_dir.
    print(f"Ολοκληρώθηκε. Δείτε τον φάκελο: {out_dir}")


In [31]:
def process_document(image_path, out_dir):
  """
  This is a placeholder function for process_document.
  Replace this with your actual implementation.

  Args:
    image_path: The path to the image file.
    out_dir: The output directory.
  """
  print(f"Processing document: {image_path} and saving output to {out_dir}")
  # Add your document processing logic here
  pass

In [32]:
if __name__ == "__main__":

    process_document('3.png', out_dir='outputs/3')
    pass


Processing document: 3.png and saving output to outputs/3
